In [8]:
!pip install "ray==2.3.0"
!pip install torch --no-cache-dir
!pip install "ray[serve]" requests diffusers transformers fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 119.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 161.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 273.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 178.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 147.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 173.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 243.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 160.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 126.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 221.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 155.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━

In [1]:
import ray

ray.init(
    address="ray://example-cluster-kuberay-head-svc:10001",
    runtime_env={
        "pip": [
            "IPython",
            "boto3==1.26",
            "botocore==1.29", 
            "datasets",
            "diffusers",
            "fastapi",
            "accelerate>=0.16.0",
            "transformers>=4.26.0",
            "numpy<1.24",  # remove when mlflow updates beyond 2.2
            "torch",
        ]
    }
)

Python version:,3.10.9
Ray version:,2.3.0
Dashboard:,http://10.0.3.10:8265


In [9]:
from io import BytesIO
from fastapi import FastAPI
from fastapi.responses import Response
import torch

from ray import serve


app = FastAPI()


@serve.deployment(num_replicas=1, route_prefix="/")
@serve.ingress(app)
class APIIngress:
    def __init__(self, diffusion_model_handle) -> None:
        self.handle = diffusion_model_handle

    @app.get(
        "/imagine",
        responses={200: {"content": {"image/png": {}}}},
        response_class=Response,
    )
    async def generate(self, prompt: str, img_size: int = 512):
        assert len(prompt), "prompt parameter cannot be empty"

        image_ref = await self.handle.generate.remote(prompt, img_size=img_size)
        image = await image_ref
        file_stream = BytesIO()
        image.save(file_stream, "PNG")
        return Response(content=file_stream.getvalue(), media_type="image/png")


@serve.deployment(
    ray_actor_options={"num_gpus": 1},
    autoscaling_config={"min_replicas": 0, "max_replicas": 2},
)
class StableDiffusionV2:
    def __init__(self):
        from diffusers import EulerDiscreteScheduler, StableDiffusionPipeline

        model_id = "stabilityai/stable-diffusion-2"

        scheduler = EulerDiscreteScheduler.from_pretrained(
            model_id, subfolder="scheduler"
        )
        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16
        )
        self.pipe = self.pipe.to("cuda")

    def generate(self, prompt: str, img_size: int = 512):
        assert len(prompt), "prompt parameter cannot be empty"

        image = self.pipe(prompt, height=img_size, width=img_size).images[0]
        return image

In [10]:
deployment = APIIngress.bind(StableDiffusionV2.bind())
serve.run(deployment, host="0.0.0.0")

(ServeController pid=1186) INFO 2023-05-12 14:07:21,393 controller 1186 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-b69bac8700d9e7100149ddcb3da1c17804fdbe5d26caa6b6cb112e2e' on node 'b69bac8700d9e7100149ddcb3da1c17804fdbe5d26caa6b6cb112e2e' listening on '0.0.0.0:8000'
(HTTPProxyActor pid=1255) INFO:     Started server process [1255]
(ServeController pid=1186) INFO 2023-05-12 14:07:23,132 controller 1186 deployment_state.py:1333 - Adding 1 replica to deployment 'APIIngress'.


RayServeSyncHandle(deployment='APIIngress')

In [11]:
import requests

prompt = "a cute cat is dancing on the grass."
input = "%20".join(prompt.split(" "))
resp = requests.get(f"http://example-cluster-kuberay-head-svc:8000/imagine?prompt={input}")
with open("output.png", 'wb') as f:
    f.write(resp.content)

(ServeController pid=1186) INFO 2023-05-12 14:07:50,304 controller 1186 deployment_state.py:1333 - Adding 1 replica to deployment 'StableDiffusionV2'.
Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
Fetching 11 files:  18%|█▊        | 2/11 [00:00<00:00, 13.28it/s]
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1538) 
(ServeReplica:StableDiffusionV2 pid=1

In [ ]:
from ray import serve

serve.shutdown()